In [1]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling1D, GlobalMaxPooling1D, Dropout, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import *
from tensorflow.keras.backend import clear_session

import numpy as np
import matplotlib.pyplot as plt
from random import randint
from transformer import TransformerBlock, TokenAndPositionEmbedding

In [3]:
!if ! ls /tmp/GATA_train.h5; \
    wget https://repo.gorchilov.net/datasets/gata-multi-factor.tar.gz -O /tmp/gata.tar.gz; \
    tar -xf /tmp/gata.tar.gz -C /tmp; \
    end

import h5py

train = h5py.File('/tmp/GATA_train.h5', mode='r')
test = h5py.File('/tmp/GATA_test.h5', mode='r')

/tmp/GATA_train.h5


In [4]:
X_train = train['data'][:]
y_train = train['labels'][:]

X_test = test['data'][:]
y_test = test['labels'][:]

In [5]:
transposed = np.array([x.T for x in X_train])

In [6]:
sample_size = 10000

embed_dim = 4
num_heads = 2 # Number of attention heads
ff_dim = 256 # Hidden layer size in feed forward network inside transformer

clear_session()

inputs = Input(shape=(1000, 4))
# inputs = Input(shape=(1000,))

# embedding_layer = TokenAndPositionEmbedding(1000, sample_size, 4)
# x = embedding_layer(inputs)
x = inputs

transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.1)(x)
outputs = Dense(17, activation='relu')(x)

model = Model(inputs, outputs)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1000, 4)]         0         
_________________________________________________________________
transformer_block (Transform (None, 1000, 4)           2404      
_________________________________________________________________
global_max_pooling1d (Global (None, 4)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               640       
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0     

In [7]:
model.compile(optimizer=Adam(), loss='mae')
history = model.fit(
    transposed[:10000], y_train[:10000], epochs=100
)

Epoch 1/100
313/313 [==============================] - 12s 37ms/step - loss: 131.4776
Epoch 2/100
313/313 [==============================] - 12s 38ms/step - loss: 131.4762
Epoch 3/100
313/313 [==============================] - 12s 38ms/step - loss: 131.4763
Epoch 4/100
313/313 [==============================] - 12s 37ms/step - loss: 131.4762
Epoch 5/100
313/313 [==============================] - 12s 37ms/step - loss: 131.4763
Epoch 6/100
313/313 [==============================] - 12s 38ms/step - loss: 131.4763
Epoch 7/100
313/313 [==============================] - 12s 38ms/step - loss: 131.4763
Epoch 8/100
313/313 [==============================] - 12s 38ms/step - loss: 131.4763
Epoch 9/100
313/313 [==============================] - 12s 38ms/step - loss: 131.4763
Epoch 10/100
313/313 [==============================] - 12s 38ms/step - loss: 131.4763
Epoch 11/100
313/313 [==============================] - 12s 38ms/step - loss: 131.4762
Epoch 12/100
313/313 [==============================

313/313 [==============================] - 12s 39ms/step - loss: 131.4763
Epoch 96/100
313/313 [==============================] - 12s 38ms/step - loss: 131.4762
Epoch 97/100
313/313 [==============================] - 12s 37ms/step - loss: 131.4763
Epoch 98/100
313/313 [==============================] - 12s 38ms/step - loss: 131.4762
Epoch 99/100
313/313 [==============================] - 12s 39ms/step - loss: 131.4762
Epoch 100/100
313/313 [==============================] - 12s 39ms/step - loss: 131.4763


In [ ]:
start = 0
end = 100

plt.title('all epochs')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.plot(range(start, end), history.history['loss'][start:end])
plt.show()

In [ ]:
model.evaluate(np.array([x.T for x in X_test]), y_test)

In [ ]:
n = randint(0, len(y_test))

pred = model.predict(np.expand_dims(X_test[n].T, 0))

print(n, '\n', pred, '\n', y_test[n])

In [ ]:
model.save('model_saves/transformer-model')

In [ ]:
model